In [7]:
import os
import random
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from captum.attr import IntegratedGradients
import math

# Set deterministic behavior for CUDA (set before torch imports)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    cudnn.deterministic = True
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    return seed

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} with {torch.cuda.device_count()} GPU(s)")

Using device: cuda with 5 GPU(s)


### Import Data

In [8]:
# Adjust the data directory as needed
DL_DIR = "../../data/deep_learning"
# Load the regression split dictionary.
with open(f'{DL_DIR}/comb_reg_dict.pkl', 'rb') as f:
    comb_reg_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_reg_dict.pkl', 'rb') as f:
    fitbit_reg_dict = pickle.load(f)

# Load the classification split dictionary.
with open(f'{DL_DIR}/comb_class_dict.pkl', 'rb') as f:
    comb_class_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_class_dict.pkl', 'rb') as f:
    fitbit_class_dict = pickle.load(f) 

### Best results Import

In [9]:
# Read in each pickle file individually (weighted)
with open("search/fitbit_reg_deep_deep_search_results.pkl", "rb") as f:
    fitbit_reg_deep_results = pickle.load(f)

with open("search/fitbit_class_deep_deep_search_results.pkl", "rb") as f:
    fitbit_class_deep_results = pickle.load(f)

with open("search/comb_reg_deep_deep_search_results.pkl", "rb") as f:
    comb_reg_deep_results = pickle.load(f)

with open("search/comb_class_deep_deep_search_results.pkl", "rb") as f:
    comb_class_deep_results = pickle.load(f)
    
    
# Read in each pickle file individually (unweighted
with open("search/fitbit_reg_deep_nw_deep_search_results.pkl", "rb") as f:
    fitbit_reg_deep_nw_results = pickle.load(f)

with open("search/fitbit_class_deep_nw_deep_search_results.pkl", "rb") as f:
    fitbit_class_deep_nw_results = pickle.load(f)

with open("search/comb_reg_deep_nw_deep_search_results.pkl", "rb") as f:
    comb_reg_deep_nw_results = pickle.load(f)

with open("search/comb_class_deep_nw_deep_search_results.pkl", "rb") as f:
    comb_class_deep_nw_results = pickle.load(f)

### Weighted

In [4]:
fitbit_reg_deep_results["optimal_configuration"]

{'value': 0.6534586661808931,
 'params': {'drop_out_prb': 0.1234638549488152,
  'lr': 1.0975595054006004e-05,
  'use_regularization': True,
  'num_epochs': 10,
  'n_filters_0': 48},
 'user_attrs': {'overall_mean_rmse': 0.6534586661808931,
  'overall_std_rmse': 0.01648839042257811,
  'bin_mean_rmse': {'1': 0.3450005299177086,
   '2': 0.7880174590693902,
   '3': 1.6688681135098693,
   '4': 2.653461974891114,
   '5': 3.7602703131715374},
  'bin_std_rmse': {'1': 0.07033304786203862,
   '2': 0.09080233647333263,
   '3': 0.12889111377806875,
   '4': 0.0939790601669137,
   '5': 0.09452101007598172}}}

In [5]:
fitbit_class_deep_results["optimal_configuration"]

{'value': 0.23325811923250495,
 'params': {'lr': 0.000565904221438412, 'n_filters_0': 24},
 'user_attrs': {'overall_acc_mean': 0.766741880767495,
  'overall_acc_std': 0.005703636802401684,
  'sensitivity_mean': 0.035102599179206564,
  'sensitivity_std': 0.03215625233012081,
  'specificity_mean': 0.9956043956043956,
  'specificity_std': 0.0035889959601218594}}

In [6]:
comb_reg_deep_results["optimal_configuration"]

{'value': 0.6447002631150113,
 'params': {'lr': 1.1216699156320343e-05,
  'dropout_prob': 0.10662333833500955,
  'num_epochs': 10,
  'n_filters_0': 56,
  'n_cov': 1,
  'use_regularization': False,
  'dilation_0': 2},
 'user_attrs': {'overall_mean_rmse': 0.6447002631150113,
  'overall_std_rmse': 0.015244285170082817,
  'bin_mean_rmse': {'1': 0.33430442785050146,
   '2': 0.7534966556675069,
   '3': 1.7620385953470161,
   '4': 2.603179206530158,
   '5': 3.620745750753796},
  'bin_std_rmse': {'1': 0.06546720474385422,
   '2': 0.06416182634837153,
   '3': 0.08561986220277645,
   '4': 0.15541045665538414,
   '5': 0.09100139025395973}}}

In [7]:
comb_class_deep_results["optimal_configuration"]

{'value': 0.2310188138995659,
 'params': {'lr': 0.00015704157833575886,
  'dropout_prob': 0.45497081070937473,
  'num_epochs': 8,
  'batch_size': 48,
  'n_filters_0': 56},
 'user_attrs': {'overall_acc_mean': 0.7689811861004341,
  'overall_acc_std': 0.006157065666334226,
  'sensitivity_mean': 0.05157318741450069,
  'sensitivity_std': 0.020542440809382948,
  'specificity_mean': 0.9934065934065934,
  'specificity_std': 0.004859523502352217}}

### Unweighted

In [5]:
fitbit_reg_deep_nw_results["optimal_configuration"]

{'value': 0.6162078713426811,
 'params': {'dropout_prob': 0.4461520705008476,
  'n_filters_0': 8,
  'num_epochs': 5},
 'user_attrs': {'overall_mean_rmse': 0.6162078713426811,
  'overall_std_rmse': 0.021693254860636075,
  'bin_mean_rmse': {'1': 0.2777239457846413,
   '2': 0.762233443104348,
   '3': 1.7028998849745907,
   '4': 2.6041541764919733,
   '5': 3.6145699780957417},
  'bin_std_rmse': {'1': 0.03366632293350492,
   '2': 0.04805768605116488,
   '3': 0.08803748023481245,
   '4': 0.09431819128946899,
   '5': 0.1769821398233237}}}

In [6]:
fitbit_class_deep_nw_results["optimal_configuration"]

{'value': 0.23437076920682842,
 'params': {'num_epochs': 9, 'lr': 0.00025685423297033865},
 'user_attrs': {'overall_acc_mean': 0.7656292307931716,
  'overall_acc_std': 0.005674767020963522,
  'sensitivity_mean': 0.03053351573187415,
  'sensitivity_std': 0.0253574253673536,
  'specificity_mean': 0.9956043956043956,
  'specificity_std': 0.005383493940182791}}

In [7]:
comb_reg_deep_nw_results["optimal_configuration"]

{'value': 0.5967643614168212,
 'params': {'lr': 0.00013555143635027785,
  'num_epochs': 10,
  'dropout_prob': 0.13884004660199356,
  'n_filters_0': 16,
  'kernel_size_0': 3,
  'n_conv': 3},
 'user_attrs': {'overall_mean_rmse': 0.5967643614168212,
  'overall_std_rmse': 0.02198441850555824,
  'bin_mean_rmse': {'1': 0.2611028774320149,
   '2': 0.7061489981085595,
   '3': 1.6480903044230897,
   '4': 2.6172805704761606,
   '5': 3.6091912234001255},
  'bin_std_rmse': {'1': 0.02387025088679633,
   '2': 0.03572499616431226,
   '3': 0.09207538787709463,
   '4': 0.1302750589462002,
   '5': 0.1720195899149569}}}

In [8]:
comb_class_deep_nw_results["optimal_configuration"]

{'value': 0.23661007453976746,
 'params': {'batch_size': 48, 'n_filters_0': 8},
 'user_attrs': {'overall_acc_mean': 0.7633899254602325,
  'overall_acc_std': 0.0034470777500334817,
  'sensitivity_mean': 0.06560875512995895,
  'sensitivity_std': 0.03019082113868996,
  'specificity_mean': 0.9816849816849818,
  'specificity_std': 0.013105160307691077}}

### Helper Functions

In [10]:
def conv_output_length(L_in, kernel_size, stride, padding, dilation):
    return (L_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

def pool_output_length(L_in, pool_kernel):
    return L_in // pool_kernel  # assume stride equals kernel size

# Build subject-level dataset (each row is one subject)
def create_subject_dataset(df, outcome_col="SI_mean", use_weights=True):
    """
    Aggregates records for each subject into a subject-level sample.
    If use_weights is False, then sample_weight is set to 1.0 for all subjects.
    """
    exclude_cols = ["PatientID", "timepoints", "si_kde_weight", "SI_mean", "is_SI", "SI_level"]
    predictor_cols = [col for col in df.columns if col not in (exclude_cols + [outcome_col])]
    
    subject_data = []
    for pid, group in df.groupby("PatientID"):
        group_sorted = group.sort_values("timepoints")
        X = group_sorted[predictor_cols].values.T  # shape: (n_features, 39)
        y = group_sorted[outcome_col].iloc[0]
        if use_weights and "si_kde_weight" in group_sorted.columns:
            weight = group_sorted["si_kde_weight"].iloc[0]
        else:
            weight = 1.0
        record = {"PatientID": pid, "X": X, outcome_col: y, "sample_weight": weight}
        if outcome_col == "is_SI" and "SI_mean" in group_sorted.columns:
            record["SI_mean"] = group_sorted["SI_mean"].iloc[0]
        subject_data.append(record)
    subj_df = pd.DataFrame(subject_data)
    subj_df[f"{outcome_col}_bin"] = np.round(subj_df[outcome_col]).astype(int)
    return subj_df, predictor_cols

def compute_rmse(y_true, y_pred):
    return math.sqrt(np.mean((y_pred - y_true)**2))

def compute_classification_metrics(y_true, y_pred):
    acc = np.mean(y_pred == y_true)
    TP = np.sum((y_pred == 1) & (y_true == 1))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
    return acc, sensitivity, specificity

def save_results_pickle(result_dict, model_name):
    save_folder = "search"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    filename = os.path.join(save_folder, f"{model_name}_deep_search_results.pkl")
    with open(filename, "wb") as f:
        pickle.dump(result_dict, f)
    print(f"Results saved to {filename}")

# Helper to compute regression performance
def compute_regression_perf(y_true, y_pred):
    perf = {}
    perf["overall"] = compute_rmse(y_true, y_pred)
    levels = [1, 2, 3, 4, 5]
    y_levels = np.round(y_true).astype(int)
    for level in levels:
        inds = np.where(y_levels == level)[0]
        if len(inds) > 0:
            perf[str(level)] = compute_rmse(y_true[inds], y_pred[inds])
        else:
            perf[str(level)] = np.nan
    return perf


def get_stratified_cv_splits(df, subject_id="PatientID", target_var="SI_mean", n_splits=5):
    """
    Performs stratified K-fold cross validation at the subject level.
    
    Parameters:
      df : pandas.DataFrame
          The original dataframe containing repeated measures.
      subject_id : str
          The column name for the subject ID (e.g., "PatientID").
      target_var : str
          The target variable; for regression use "SI_mean" and for classification use "is_SI".
      n_splits : int
          Number of folds for cross validation.
    
    Returns:
      splits : list of tuples
          A list where each element is a tuple (train_df, test_df) corresponding
          to one fold. Each dataframe contains all rows (i.e. repeated measures) for the patients in that fold.
    
    Behavior:
      - Isolates unique patient IDs and their target variable by dropping duplicates.
      - If target_var is "SI_mean", creates a new column "SI_mean_levels" (rounded SI_mean).
      - Uses the resulting column as the stratification column.
      - Performs stratified K-fold CV and then subsets the original dataframe based on the patient IDs.
    """
    # Create a subject-level dataframe (unique patient IDs with their target variable)
    subject_df = df[[subject_id, target_var]].drop_duplicates(subset=[subject_id]).copy()
    
    # For regression: create a new column with the rounded SI_mean values.
    if target_var == "SI_mean":
        subject_df["SI_mean_levels"] = subject_df[target_var].round().astype(int)
        strat_col = "SI_mean_levels"
    else:
        strat_col = target_var  # For classification, use the target directly.
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    splits = []
    
    # Get the subject IDs and stratification labels
    subjects = subject_df[subject_id].values
    strat_labels = subject_df[strat_col].values
    
    # For each fold, retrieve patient IDs and then subset the original dataframe.
    for train_idx, test_idx in skf.split(subjects, strat_labels):
        train_patient_ids = subject_df.iloc[train_idx][subject_id].values
        test_patient_ids  = subject_df.iloc[test_idx][subject_id].values
        train_split = df[df[subject_id].isin(train_patient_ids)]
        test_split  = df[df[subject_id].isin(test_patient_ids)]
        splits.append((train_split, test_split))
    
    return splits


### Model Function Definitions

In [11]:
############################################
# Regression: Fitbit Regression using Integrated Gradients
def run_fitbit_regression_best(fitbit_reg_dict):
    set_seed(42)
    # Updated hyperparameters from deep search for fitbit_reg:
    params = {
        "batch_size": 32,
        "num_epochs": 10,
        "lr": 1.0975595054006004e-05,
        "use_regularization": True,
        "n_conv": 3,
        "n_hidden": 2,
        "n_filters_0": 48,
        "kernel_size_0": 7,
        "dropout_prob": 0.1234638549488152,
        "use_pool_0": True,
        "stride_0": 1,
        "dilation_0": 1
    }
    
    # Create subject-level datasets from full train and test splits.
    train_df, predictor_cols = create_subject_dataset(fitbit_reg_dict['train'], outcome_col="SI_mean")
    test_df, _ = create_subject_dataset(fitbit_reg_dict['test'], outcome_col="SI_mean")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    sw_train = train_df["sample_weight"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = test_df["sample_weight"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    # Build convolutional network.
    kernel_size = params["kernel_size_0"]
    stride = params["stride_0"]
    dilation = params["dilation_0"]
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, 
                     out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, 
                     stride=stride, 
                     dilation=dilation, 
                     padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    if params["use_pool_0"]:
        layers.append(nn.MaxPool1d(kernel_size=2))
        conv_out_len = pool_output_length(seq_len, 2)
    else:
        conv_out_len = seq_len
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_out_len
    
    # Build fully connected network.
    fc_layers = []
    in_features = flattened_dim
    for _ in range(params["n_hidden"]):
        fc_layers.append(nn.Linear(in_features, 64))
        fc_layers.append(nn.ReLU())
        fc_layers.append(nn.Dropout(params["dropout_prob"]))
        in_features = 64
    fc_layers.append(nn.Linear(in_features, 1))
    fc_net = nn.Sequential(*fc_layers)
    
    class FitRegCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(FitRegCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = FitRegCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction="none")
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    # Train the model on the full training set.
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    # Helper function to get predictions.
    def get_preds(loader, model):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds
    
    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION (CV) on the training data for regression
    cv_splits = get_stratified_cv_splits(fitbit_reg_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []  # to store each fold's performance dictionary
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        # Create subject-level datasets for this CV fold.
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = cv_train_df["sample_weight"].values.astype(np.float32)
        
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = cv_val_df["sample_weight"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        # Build a new model instance with the same architecture.
        kernel_size = params["kernel_size_0"]
        stride = params["stride_0"]
        dilation = params["dilation_0"]
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv, 
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, 
                            stride=stride, 
                            dilation=dilation, 
                            padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        if params["use_pool_0"]:
            layers_cv.append(nn.MaxPool1d(kernel_size=2))
            conv_out_len_cv = pool_output_length(seq_len_cv, 2)
        else:
            conv_out_len_cv = seq_len_cv
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_out_len_cv
        
        fc_layers_cv = []
        in_features_cv = flattened_dim_cv
        for _ in range(params["n_hidden"]):
            fc_layers_cv.append(nn.Linear(in_features_cv, 64))
            fc_layers_cv.append(nn.ReLU())
            fc_layers_cv.append(nn.Dropout(params["dropout_prob"]))
            in_features_cv = 64
        fc_layers_cv.append(nn.Linear(in_features_cv, 1))
        fc_net_cv = nn.Sequential(*fc_layers_cv)
        
        class FitRegCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitRegCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = FitRegCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction="none")
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        # Compute overall MSE for this fold.
        mse_cv = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        fold_perf["mse"] = mse_cv
        cv_fold_metrics.append(fold_perf)
    
    # Aggregate CV metrics over folds (including mse).
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_regression", "fitbit_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df


############################################
# Regression: Comb Regression using Integrated Gradients
def run_comb_regression_best(comb_dict):
    set_seed(42)
    # Updated hyperparameters from deep search for comb_reg:
    params = {
        "lr": 1.1216699156320343e-05,
        "dropout_prob": 0.10662333833500955,
        "num_epochs": 10,
        "n_filters_0": 56,
        "n_cov": 1,
        "use_regularization": False,
        "dilation_0": 2,
        "use_pool_0": True
    }
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="SI_mean")
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="SI_mean")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    sw_train = train_df["sample_weight"].values.astype(np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = test_df["sample_weight"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Build convolutional network.
    kernel_size = 3
    stride = 1
    dilation = params["dilation_0"]
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels,
                     out_channels=params["n_filters_0"],
                     kernel_size=kernel_size,
                     stride=stride,
                     dilation=dilation,
                     padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    if params["use_pool_0"]:
        layers.append(nn.MaxPool1d(kernel_size=2))
        conv_out_len = pool_output_length(seq_len, 2)
    else:
        conv_out_len = seq_len
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_out_len
    
    # For comb regression, a direct mapping is used.
    fc_net = nn.Linear(flattened_dim, 1)
    
    class CombRegCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(CombRegCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = CombRegCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction='none')
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for comb regression
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = cv_train_df["sample_weight"].values.astype(np.float32)
        
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = cv_val_df["sample_weight"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = 3
        stride = 1
        dilation = params["dilation_0"]
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv,
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size,
                            stride=stride,
                            dilation=dilation,
                            padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        if params["use_pool_0"]:
            layers_cv.append(nn.MaxPool1d(kernel_size=2))
            conv_out_len_cv = pool_output_length(seq_len_cv, 2)
        else:
            conv_out_len_cv = seq_len_cv
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_out_len_cv
        
        fc_net_cv = nn.Linear(flattened_dim_cv, 1)
        
        class CombRegCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombRegCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = CombRegCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction='none')
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        # Compute overall MSE for this fold.
        mse_cv = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        fold_perf["mse"] = mse_cv
        cv_fold_metrics.append(fold_perf)
    
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_regression", "comb_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df


############################################
# Classification: Comb Classification using Integrated Gradients
def run_comb_classification_best(comb_dict):
    set_seed(42)
    # Updated hyperparameters from deep search for comb_class:
    params = {
        "lr": 0.00015704157833575886,
        "dropout_prob": 0.45497081070937473,
        "num_epochs": 8,
        "batch_size": 48,
        "n_filters_0": 56
    }
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="is_SI")
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="is_SI")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    kernel_size = 3
    stride = 1
    dilation = 1
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_output_length(seq_len, kernel_size, stride, padding, dilation)
    
    fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
    
    class CombClassCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(CombClassCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = CombClassCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    train_auc = roc_auc_score(y_train_true, y_train_pred)
    test_auc = roc_auc_score(y_test_true, y_test_pred)
    y_train_bin = (y_train_pred >= 0.5).astype(np.float32)
    y_test_bin = (y_test_pred >= 0.5).astype(np.float32)
    TP_train = np.sum((y_train_bin == 1) & (y_train_true == 1))
    FN_train = np.sum((y_train_bin == 0) & (y_train_true == 1))
    TN_train = np.sum((y_train_bin == 0) & (y_train_true == 0))
    FP_train = np.sum((y_train_bin == 1) & (y_train_true == 0))
    train_acc = np.mean(y_train_bin == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_bin == 1) & (y_test_true == 1))
    FN_test = np.sum((y_test_bin == 0) & (y_test_true == 1))
    TN_test = np.sum((y_test_bin == 0) & (y_test_true == 0))
    FP_test = np.sum((y_test_bin == 1) & (y_test_true == 0))
    test_acc = np.mean(y_test_bin == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan

    perf_train_df = pd.DataFrame({
        "model": ["comb_classification"],
        "type": ["train"],
        "AUC": [train_auc],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_classification"],
        "type": ["test"],
        "AUC": [test_auc],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for comb classification
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = 3
        stride = 1
        dilation = 1
        padding = ((kernel_size - 1) // 2)* dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv,
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_output_length(seq_len_cv, kernel_size, stride, padding, dilation)
        
        fc_net_cv = nn.Sequential(nn.Linear(flattened_dim_cv, 1))
        
        class CombClassCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombClassCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = CombClassCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        # Compute binary cross entropy (with epsilon for numerical stability)
        epsilon = 1e-12
        bce_cv = np.mean(- (y_val_true_cv * np.log(y_val_pred_cv + epsilon) +
                            (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon)))
        cv_fold_metrics.append({
            "AUC": auc_cv,
            "accuracy": acc,
            "sensitivity": sens,
            "specificity": spec,
            "bce": bce_cv
        })
    
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_classification", "comb_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bce": [mean_metrics["bce"], sd_metrics["bce"]]
    })
    
    return performance_df, shap_df, cv_val_df


############################################
# Classification: Fitbit Classification using Integrated Gradients
def run_fitbit_classification_best(fitbit_dict):
    set_seed(42)
    # Updated hyperparameters from deep search for fitbit_class:
    params = {
        "lr": 0.000565904221438412,
        "dropout_prob": 0.45160954145010296,
        "num_epochs": 8,
        "n_filters_0": 24,
        "kernel_size_0": 3
    }
    train_df, predictor_cols = create_subject_dataset(fitbit_dict['train'], outcome_col="is_SI")
    test_df, _ = create_subject_dataset(fitbit_dict['test'], outcome_col="is_SI")
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    kernel_size = params["kernel_size_0"]
    stride = 1
    dilation = 1
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_output_length(seq_len, kernel_size, stride, padding, dilation)
    
    fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
    
    class FitbitClassCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(FitbitClassCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = FitbitClassCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader, model):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader, model)
    y_test_true, y_test_pred = get_preds(test_loader, model)
    
    train_auc = roc_auc_score(y_train_true, y_train_pred)
    test_auc = roc_auc_score(y_test_true, y_test_pred)
    y_train_bin = (y_train_pred >= 0.5).astype(np.float32)
    y_test_bin = (y_test_pred >= 0.5).astype(np.float32)
    TP_train = np.sum((y_train_bin == 1) & (y_train_true == 1))
    FN_train = np.sum((y_train_bin == 0) & (y_train_true == 1))
    TN_train = np.sum((y_train_bin == 0) & (y_train_true == 0))
    FP_train = np.sum((y_train_bin == 1) & (y_train_true == 0))
    train_acc = np.mean(y_train_bin == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_bin == 1) & (y_test_true == 1))
    FN_test = np.sum((y_test_bin == 0) & (y_test_true == 1))
    TN_test = np.sum((y_test_bin == 0) & (y_test_true == 0))
    FP_test = np.sum((y_test_bin == 1) & (y_test_true == 0))
    test_acc = np.mean(y_test_bin == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_classification"],
        "type": ["train"],
        "AUC": [train_auc],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_classification"],
        "type": ["test"],
        "AUC": [test_auc],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for fitbit classification
    cv_splits = get_stratified_cv_splits(fitbit_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI")
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI")
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = params["kernel_size_0"]
        stride = 1
        dilation = 1
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv, out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_output_length(seq_len_cv, kernel_size, stride, padding, dilation)
        
        fc_net_cv = nn.Sequential(nn.Linear(flattened_dim_cv, 1))
        
        class FitbitClassCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitbitClassCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = FitbitClassCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        y_val_true_cv, y_val_pred_cv = get_preds(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        epsilon = 1e-12
        bce_cv = np.mean(- (y_val_true_cv * np.log(y_val_pred_cv + epsilon) +
                            (1 - y_val_true_cv) * np.log(1 - y_val_pred_cv + epsilon)))
        cv_fold_metrics.append({
            "AUC": auc_cv,
            "accuracy": acc,
            "sensitivity": sens,
            "specificity": spec,
            "bce": bce_cv
        })
    
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_classification", "fitbit_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bce": [mean_metrics["bce"], sd_metrics["bce"]]
    })
    
    return performance_df, shap_df, cv_val_df


### Running the Models and Displaying Results

In [12]:
# Run the functions (each returns a tuple: (performance_df, shap_df))
perf_fitbit_reg, shap_fitbit_reg, cv_fitbit_reg = run_fitbit_regression_best(fitbit_reg_dict)
perf_fitbit_class, shap_fitbit_class, cv_fitbit_class = run_fitbit_classification_best(fitbit_class_dict)
perf_comb_reg, shap_comb_reg, cv_comb_reg = run_comb_regression_best(comb_reg_dict)
perf_comb_class, shap_comb_class, cv_comb_class = run_comb_classification_best(comb_class_dict)

In [14]:
# Combine the performance dataframes for regression and classification
# best_perf_reg = pd.concat([perf_fitbit_reg, perf_comb_reg])
# best_perf_class = pd.concat([perf_fitbit_class, perf_comb_class])


# Save the combined performance dataframes as TSV files (without index)
# best_perf_reg.to_csv("results/best_perf_reg.tsv", sep="\t", index=False)
# best_perf_class.to_csv("results/best_perf_class.tsv", sep="\t", index=False)

# Save the SHAP score dataframes separately as TSV files (without index)
shap_fitbit_reg.to_csv("results/shap_fitbit_reg.tsv", sep="\t", index=False)
shap_fitbit_class.to_csv("results/shap_fitbit_class.tsv", sep="\t", index=False)
shap_comb_reg.to_csv("results/shap_comb_reg.tsv", sep="\t", index=False)
shap_comb_class.to_csv("results/shap_comb_class.tsv", sep="\t", index=False)

### Unweighted

In [13]:
# Non‐Weighted Fitbit Regression
def run_fitbit_regression_nw_best(fitbit_reg_dict):
    set_seed(42)
    # Updated hyperparameters for non‐weighted fitbit regression
    params = {
        "batch_size": 32,                      
        "num_epochs": 5,                        
        "lr": 1.0975595054006004e-05,           
        "use_regularization": True,            
        "n_conv": 3,                           
        "n_hidden": 2,                          
        "n_filters_0": 8,                      
        "kernel_size_0": 7,                    
        "dropout_prob": 0.4461520705008476,      
        "use_pool_0": True,                    
        "stride_0": 1,                          
        "dilation_0": 1                        
    }
    
    # Create subject-level datasets without using sample weights.
    train_df, predictor_cols = create_subject_dataset(fitbit_reg_dict['train'], outcome_col="SI_mean", use_weights=False)
    test_df, _ = create_subject_dataset(fitbit_reg_dict['test'], outcome_col="SI_mean", use_weights=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    # All weights are 1.0
    sw_train = np.ones_like(y_train, dtype=np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = np.ones_like(y_test, dtype=np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    # Build convolutional network.
    kernel_size = params["kernel_size_0"]
    stride = params["stride_0"]
    dilation = params["dilation_0"]
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, 
                     out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, 
                     stride=stride, 
                     dilation=dilation, 
                     padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    if params["use_pool_0"]:
        layers.append(nn.MaxPool1d(kernel_size=2))
        conv_out_len = pool_output_length(seq_len, 2)
    else:
        conv_out_len = seq_len
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_out_len
    
    # Build fully connected network.
    fc_layers = []
    in_features = flattened_dim
    for _ in range(params["n_hidden"]):
        fc_layers.append(nn.Linear(in_features, 64))
        fc_layers.append(nn.ReLU())
        fc_layers.append(nn.Dropout(params["dropout_prob"]))
        in_features = 64
    fc_layers.append(nn.Linear(in_features, 1))
    fc_net = nn.Sequential(*fc_layers)
    
    class FitRegCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(FitRegCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = FitRegCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction="none")
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    # Train the model.
    model.train()
    for epoch in range(params["num_epochs"]):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    # Helper to get predictions.
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_nw_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_nw_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight Fitbit regression
    cv_splits = get_stratified_cv_splits(fitbit_reg_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean", use_weights=False)
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean", use_weights=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = np.ones_like(y_cv_train, dtype=np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = np.ones_like(y_cv_val, dtype=np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        
        # Build a new model instance (replicating architecture and hyperparameters)
        kernel_size = params["kernel_size_0"]
        stride = params["stride_0"]
        dilation = params["dilation_0"]
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv, 
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, 
                            stride=stride, 
                            dilation=dilation, 
                            padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        if params["use_pool_0"]:
            layers_cv.append(nn.MaxPool1d(kernel_size=2))
            conv_out_len_cv = pool_output_length(seq_len_cv, 2)
        else:
            conv_out_len_cv = seq_len_cv
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_out_len_cv
        
        fc_layers_cv = []
        in_features_cv = flattened_dim_cv
        for _ in range(params["n_hidden"]):
            fc_layers_cv.append(nn.Linear(in_features_cv, 64))
            fc_layers_cv.append(nn.ReLU())
            fc_layers_cv.append(nn.Dropout(params["dropout_prob"]))
            in_features_cv = 64
        fc_layers_cv.append(nn.Linear(in_features_cv, 1))
        fc_net_cv = nn.Sequential(*fc_layers_cv)
        
        class FitRegCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitRegCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = FitRegCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction="none")
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        # Evaluation on CV validation set
        model_cv.eval()
        def get_preds_cv(loader, model_obj):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch, _ in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_obj(X_batch)
                    preds.append(outputs.cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv, model_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        # Compute overall MSE for the CV fold.
        fold_mse = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf["mse"] = fold_mse
        cv_fold_metrics.append(fold_perf)
    
    # Aggregate fold metrics.
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_nw_regression", "fitbit_nw_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df

################################################################
# Non‐Weighted Fitbit Classification
def run_fitbit_classification_nw_best(fitbit_dict):
    set_seed(42)
    # Updated hyperparameters for non‐weighted fitbit classification.
    # Only num_epochs and lr are updated.
    params = {
        "lr": 0.00025685423297033865,        
        "dropout_prob": 0.45160954145010296,    
        "num_epochs": 9,                       
        "n_filters_0": 24,                      
        "kernel_size_0": 3                      
    }
    # Create subject-level datasets without using sample weights.
    train_df, predictor_cols = create_subject_dataset(fitbit_dict['train'], outcome_col="is_SI", use_weights=False)
    test_df, _ = create_subject_dataset(fitbit_dict['test'], outcome_col="is_SI", use_weights=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    kernel_size = params["kernel_size_0"]
    stride = 1
    dilation = 1
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_output_length(seq_len, kernel_size, stride, padding, dilation)
    
    fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
    
    class FitbitClassCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(FitbitClassCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = FitbitClassCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    train_auc = roc_auc_score(y_train_true, y_train_pred)
    test_auc = roc_auc_score(y_test_true, y_test_pred)
    y_train_bin = (y_train_pred >= 0.5).astype(np.float32)
    y_test_bin = (y_test_pred >= 0.5).astype(np.float32)
    TP_train = np.sum((y_train_bin == 1) & (y_train_true == 1))
    FN_train = np.sum((y_train_bin == 0) & (y_train_true == 1))
    TN_train = np.sum((y_train_bin == 0) & (y_train_true == 0))
    FP_train = np.sum((y_train_bin == 1) & (y_train_true == 0))
    train_acc = np.mean(y_train_bin == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan

    TP_test = np.sum((y_test_bin == 1) & (y_test_true == 1))
    FN_test = np.sum((y_test_bin == 0) & (y_test_true == 1))
    TN_test = np.sum((y_test_bin == 0) & (y_test_true == 0))
    FP_test = np.sum((y_test_bin == 1) & (y_test_true == 0))
    test_acc = np.mean(y_test_bin == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["fitbit_nw_classification"],
        "type": ["train"],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["fitbit_nw_classification"],
        "type": ["test"],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight fitbit classification
    cv_splits = get_stratified_cv_splits(fitbit_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI", use_weights=False)
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI", use_weights=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = params["kernel_size_0"]
        stride = 1
        dilation = 1
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv,
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_output_length(seq_len_cv, kernel_size, stride, padding, dilation)
        
        fc_net_cv = nn.Sequential(nn.Linear(flattened_dim_cv, 1))
        
        class FitbitClassCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(FitbitClassCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = FitbitClassCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        def get_preds_cv(loader, model_obj):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_obj(X_batch)
                    preds.append(torch.sigmoid(outputs).cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        # Compute binary cross entropy for the CV fold.
        eps = 1e-8
        bce_cv = np.mean(- (y_val_true_cv * np.log(np.clip(y_val_pred_cv, eps, 1 - eps)) + 
                            (1 - y_val_true_cv) * np.log(np.clip(1 - y_val_pred_cv, eps, 1 - eps))))
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bce": bce_cv})
    
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["fitbit_nw_classification", "fitbit_nw_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bce": [mean_metrics["bce"], sd_metrics["bce"]]
    })
    
    return performance_df, shap_df, cv_val_df

################################################################
# Non‐Weighted Comb Regression
def run_comb_regression_nw_best(comb_dict):
    set_seed(42)
    # Updated hyperparameters for non‐weighted comb regression.
    params = {
        "lr": 0.00013555143635027785,         
        "dropout_prob": 0.13884004660199356,     
        "num_epochs": 10,                       
        "n_filters_0": 16,                      
        "kernel_size_0": 3,                     
        "n_conv": 3,                           
        "n_cov": 1,                            
        "use_regularization": False,          
        "dilation_0": 2,                      
        "use_pool_0": True                    
    }
    # Create subject-level datasets without using sample weights.
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="SI_mean", use_weights=False)
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="SI_mean", use_weights=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["SI_mean"].values.astype(np.float32)
    sw_train = np.ones_like(y_train, dtype=np.float32)
    
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["SI_mean"].values.astype(np.float32)
    sw_test = np.ones_like(y_test, dtype=np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape

    # Build convolutional network.
    kernel_size = params["kernel_size_0"]
    stride = 1
    dilation = params["dilation_0"]
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels,
                     out_channels=params["n_filters_0"],
                     kernel_size=kernel_size,
                     stride=stride,
                     dilation=dilation,
                     padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    if params["use_pool_0"]:
        layers.append(nn.MaxPool1d(kernel_size=2))
        conv_out_len = pool_output_length(seq_len, 2)
    else:
        conv_out_len = seq_len
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_out_len
    
    # For comb regression, a direct mapping is used.
    fc_net = nn.Linear(flattened_dim, 1)
    
    class CombRegCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(CombRegCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = CombRegCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.MSELoss(reduction='none')
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1),
        torch.tensor(sw_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch, w_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            w_batch = w_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = (loss_fn(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch, _ in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(outputs.cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    perf_train = compute_regression_perf(y_train_true, y_train_pred)
    perf_test  = compute_regression_perf(y_test_true, y_test_pred)
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_nw_regression"],
        "type": ["train"],
        "1": [perf_train["1"]],
        "2": [perf_train["2"]],
        "3": [perf_train["3"]],
        "4": [perf_train["4"]],
        "5": [perf_train["5"]],
        "overall": [perf_train["overall"]]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_nw_regression"],
        "type": ["test"],
        "1": [perf_test["1"]],
        "2": [perf_test["2"]],
        "3": [perf_test["3"]],
        "4": [perf_test["4"]],
        "5": [perf_test["5"]],
        "overall": [perf_test["overall"]]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight comb regression
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="SI_mean", use_weights=False)
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="SI_mean", use_weights=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["SI_mean"].values.astype(np.float32)
        sw_cv_train = np.ones_like(y_cv_train, dtype=np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["SI_mean"].values.astype(np.float32)
        sw_cv_val = np.ones_like(y_cv_val, dtype=np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = params["kernel_size_0"]
        stride = 1
        dilation = params["dilation_0"]
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv,
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size,
                            stride=stride,
                            dilation=dilation,
                            padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        if params["use_pool_0"]:
            layers_cv.append(nn.MaxPool1d(kernel_size=2))
            conv_out_len_cv = pool_output_length(seq_len_cv, 2)
        else:
            conv_out_len_cv = seq_len_cv
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_out_len_cv
        
        fc_net_cv = nn.Linear(flattened_dim_cv, 1)
        
        class CombRegCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombRegCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = CombRegCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.MSELoss(reduction='none')
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1),
            torch.tensor(sw_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=32, shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=32, shuffle=False)
        
        model_cv.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, w_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                w_batch = w_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = (loss_fn_cv(outputs, y_batch).view(-1) * w_batch.view(-1)).mean()
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        def get_preds_cv(loader, model_obj):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch, _ in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_obj(X_batch)
                    preds.append(outputs.cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv, model_cv)
        fold_perf = compute_regression_perf(y_val_true_cv, y_val_pred_cv)
        fold_mse = np.mean((y_val_true_cv - y_val_pred_cv) ** 2)
        fold_perf["mse"] = fold_mse
        cv_fold_metrics.append(fold_perf)
    
    keys = ["1", "2", "3", "4", "5", "overall", "mse"]
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in keys}
    sd_metrics   = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in keys}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_nw_regression", "comb_nw_regression"],
        "1": [mean_metrics["1"], sd_metrics["1"]],
        "2": [mean_metrics["2"], sd_metrics["2"]],
        "3": [mean_metrics["3"], sd_metrics["3"]],
        "4": [mean_metrics["4"], sd_metrics["4"]],
        "5": [mean_metrics["5"], sd_metrics["5"]],
        "overall": [mean_metrics["overall"], sd_metrics["overall"]],
        "mse": [mean_metrics["mse"], sd_metrics["mse"]]
    })
    
    return performance_df, shap_df, cv_val_df

################################################################
# Non‐Weighted Comb Classification
def run_comb_classification_nw_best(comb_dict):
    set_seed(42)
    # Updated hyperparameters for non‐weighted comb classification.
    # Only batch_size and n_filters_0 are updated.
    params = {
        "lr": 0.00015704157833575886,        
        "dropout_prob": 0.45497081070937473,    
        "num_epochs": 8,                       
        "batch_size": 48,                      
        "n_filters_0": 8                       
    }
    # Create subject-level datasets without using sample weights.
    train_df, predictor_cols = create_subject_dataset(comb_dict['train'], outcome_col="is_SI", use_weights=False)
    test_df, _ = create_subject_dataset(comb_dict['test'], outcome_col="is_SI", use_weights=False)
    
    X_train = np.stack(train_df["X"].values, axis=0)
    y_train = train_df["is_SI"].values.astype(np.float32)
    X_test = np.stack(test_df["X"].values, axis=0)
    y_test = test_df["is_SI"].values.astype(np.float32)
    
    n_subjects, input_channels, seq_len = X_train.shape
    
    kernel_size = 3
    stride = 1
    dilation = 1
    padding = ((kernel_size - 1) // 2) * dilation
    conv = nn.Conv1d(in_channels=input_channels, out_channels=params["n_filters_0"],
                     kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
    relu = nn.ReLU()
    layers = [conv, relu, nn.Dropout(params["dropout_prob"])]
    conv_net = nn.Sequential(*layers)
    flattened_dim = params["n_filters_0"] * conv_output_length(seq_len, kernel_size, stride, padding, dilation)
    
    fc_net = nn.Sequential(nn.Linear(flattened_dim, 1))
    
    class CombClassCNN(nn.Module):
        def __init__(self, conv_net, fc_net):
            super(CombClassCNN, self).__init__()
            self.conv = conv_net
            self.fc = fc_net
        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)
            return self.fc(x)
    
    model = CombClassCNN(conv_net, fc_net).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])
    loss_fn = nn.BCEWithLogitsLoss()
    num_epochs = params["num_epochs"]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    def get_preds(loader):
        preds, truths = [], []
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                preds.append(torch.sigmoid(outputs).cpu().numpy())
                truths.append(y_batch.cpu().numpy())
        preds = np.concatenate(preds).flatten()
        truths = np.concatenate(truths).flatten()
        return truths, preds

    model.eval()
    y_train_true, y_train_pred = get_preds(train_loader)
    y_test_true, y_test_pred = get_preds(test_loader)
    
    TP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 1))
    FN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 1))
    TN_train = np.sum((y_train_pred < 0.5) & (y_train_true == 0))
    FP_train = np.sum((y_train_pred >= 0.5) & (y_train_true == 0))
    train_acc = np.mean((y_train_pred >= 0.5) == y_train_true)
    train_sens = TP_train / (TP_train + FN_train) if (TP_train + FN_train) > 0 else np.nan
    train_spec = TN_train / (TN_train + FP_train) if (TN_train + FP_train) > 0 else np.nan
    
    TP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 1))
    FN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 1))
    TN_test = np.sum((y_test_pred < 0.5) & (y_test_true == 0))
    FP_test = np.sum((y_test_pred >= 0.5) & (y_test_true == 0))
    test_acc = np.mean((y_test_pred >= 0.5) == y_test_true)
    test_sens = TP_test / (TP_test + FN_test) if (TP_test + FN_test) > 0 else np.nan
    test_spec = TN_test / (TN_test + FP_test) if (TN_test + FP_test) > 0 else np.nan
    
    perf_train_df = pd.DataFrame({
        "model": ["comb_nw_classification"],
        "type": ["train"],
        "accuracy": [train_acc],
        "sensitivity": [train_sens],
        "specificity": [train_spec]
    })
    perf_test_df = pd.DataFrame({
        "model": ["comb_nw_classification"],
        "type": ["test"],
        "accuracy": [test_acc],
        "sensitivity": [test_sens],
        "specificity": [test_spec]
    })
    performance_df = pd.concat([perf_train_df, perf_test_df], ignore_index=True)
    
    # Compute Integrated Gradients for SHAP.
    set_seed(42)
    model_for_attr = model.module if hasattr(model, 'module') else model
    ig = IntegratedGradients(model_for_attr)
    input_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    baseline = torch.zeros_like(input_tensor)
    attributions = ig.attribute(input_tensor, baselines=baseline, n_steps=50)
    mean_attr = attributions.mean(dim=0).mean(dim=1).cpu().detach().numpy()
    sd_attr = attributions.abs().std(dim=0).mean(dim=1).cpu().detach().numpy()
    if len(predictor_cols) != len(mean_attr):
        min_len = min(len(predictor_cols), len(mean_attr))
        predictor_names = predictor_cols[:min_len]
        mean_attr = mean_attr[:min_len]
        sd_attr = sd_attr[:min_len]
    else:
        predictor_names = predictor_cols
    shap_df = pd.DataFrame({
        "predictor": predictor_names,
        "mean_abs_integrated_gradients": mean_attr,
        "sd_abs_integrated_gradients": sd_attr
    })
    
    #################################################################
    # CROSS-VALIDATION for non-weight comb classification
    cv_splits = get_stratified_cv_splits(comb_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    cv_fold_metrics = []
    
    for fold, (cv_train_raw, cv_val_raw) in enumerate(cv_splits, start=1):
        cv_train_df, _ = create_subject_dataset(cv_train_raw, outcome_col="is_SI", use_weights=False)
        cv_val_df, _ = create_subject_dataset(cv_val_raw, outcome_col="is_SI", use_weights=False)
        X_cv_train = np.stack(cv_train_df["X"].values, axis=0)
        y_cv_train = cv_train_df["is_SI"].values.astype(np.float32)
        X_cv_val = np.stack(cv_val_df["X"].values, axis=0)
        y_cv_val = cv_val_df["is_SI"].values.astype(np.float32)
        
        n_cv, input_channels_cv, seq_len_cv = X_cv_train.shape
        kernel_size = 3
        stride = 1
        dilation = 1
        padding = ((kernel_size - 1) // 2) * dilation
        conv_cv = nn.Conv1d(in_channels=input_channels_cv,
                            out_channels=params["n_filters_0"],
                            kernel_size=kernel_size, stride=stride, dilation=dilation, padding=padding)
        relu_cv = nn.ReLU()
        layers_cv = [conv_cv, relu_cv, nn.Dropout(params["dropout_prob"])]
        conv_net_cv = nn.Sequential(*layers_cv)
        flattened_dim_cv = params["n_filters_0"] * conv_output_length(seq_len_cv, kernel_size, stride, padding, dilation)
        
        fc_net_cv = nn.Sequential(nn.Linear(flattened_dim_cv, 1))
        
        class CombClassCNN_CV(nn.Module):
            def __init__(self, conv_net, fc_net):
                super(CombClassCNN_CV, self).__init__()
                self.conv = conv_net
                self.fc = fc_net
            def forward(self, x):
                x = self.conv(x)
                x = x.view(x.size(0), -1)
                return self.fc(x)
        
        model_cv = CombClassCNN_CV(conv_net_cv, fc_net_cv).to(device)
        if torch.cuda.device_count() > 1:
            model_cv = nn.DataParallel(model_cv)
        optimizer_cv = optim.Adam(model_cv.parameters(), lr=params["lr"])
        loss_fn_cv = nn.BCEWithLogitsLoss()
        
        train_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_train, dtype=torch.float32),
            torch.tensor(y_cv_train, dtype=torch.float32).view(-1, 1)
        )
        val_dataset_cv = torch.utils.data.TensorDataset(
            torch.tensor(X_cv_val, dtype=torch.float32),
            torch.tensor(y_cv_val, dtype=torch.float32).view(-1, 1)
        )
        train_loader_cv = torch.utils.data.DataLoader(train_dataset_cv, batch_size=params["batch_size"], shuffle=True)
        val_loader_cv = torch.utils.data.DataLoader(val_dataset_cv, batch_size=params["batch_size"], shuffle=False)
        
        model_cv.train()
        for epoch in range(params["num_epochs"]):
            for X_batch, y_batch in train_loader_cv:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer_cv.zero_grad()
                outputs = model_cv(X_batch)
                loss = loss_fn_cv(outputs, y_batch)
                loss.backward()
                optimizer_cv.step()
        
        model_cv.eval()
        def get_preds_cv(loader, model_obj):
            preds, truths = [], []
            with torch.no_grad():
                for X_batch, y_batch in loader:
                    X_batch = X_batch.to(device)
                    outputs = model_obj(X_batch)
                    preds.append(torch.sigmoid(outputs).cpu().numpy())
                    truths.append(y_batch.cpu().numpy())
            preds = np.concatenate(preds).flatten()
            truths = np.concatenate(truths).flatten()
            return truths, preds
        y_val_true_cv, y_val_pred_cv = get_preds_cv(val_loader_cv, model_cv)
        auc_cv = roc_auc_score(y_val_true_cv, y_val_pred_cv)
        y_val_bin = (y_val_pred_cv >= 0.5).astype(np.float32)
        TP = np.sum((y_val_bin == 1) & (y_val_true_cv == 1))
        FN = np.sum((y_val_bin == 0) & (y_val_true_cv == 1))
        TN = np.sum((y_val_bin == 0) & (y_val_true_cv == 0))
        FP = np.sum((y_val_bin == 1) & (y_val_true_cv == 0))
        acc = np.mean(y_val_bin == y_val_true_cv)
        sens = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        spec = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        eps = 1e-8
        bce_cv = np.mean(- (y_val_true_cv * np.log(np.clip(y_val_pred_cv, eps, 1 - eps)) + 
                            (1 - y_val_true_cv) * np.log(np.clip(1 - y_val_pred_cv, eps, 1 - eps))))
        cv_fold_metrics.append({"AUC": auc_cv, "accuracy": acc, "sensitivity": sens, "specificity": spec, "bce": bce_cv})
    
    mean_metrics = {k: np.mean([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    sd_metrics = {k: np.std([fold[k] for fold in cv_fold_metrics]) for k in ["AUC", "accuracy", "sensitivity", "specificity", "bce"]}
    cv_val_df = pd.DataFrame({
        "stat": ["mean", "sd"],
        "model": ["comb_nw_classification", "comb_nw_classification"],
        "AUC": [mean_metrics["AUC"], sd_metrics["AUC"]],
        "accuracy": [mean_metrics["accuracy"], sd_metrics["accuracy"]],
        "sensitivity": [mean_metrics["sensitivity"], sd_metrics["sensitivity"]],
        "specificity": [mean_metrics["specificity"], sd_metrics["specificity"]],
        "bce": [mean_metrics["bce"], sd_metrics["bce"]]
    })
    
    return performance_df, shap_df, cv_val_df


In [14]:
perf_fitbit_reg_nw, shap_fitbit_reg_nw, cv_fitbit_reg_nw = run_fitbit_regression_nw_best(fitbit_reg_dict)
perf_fitbit_class_nw, shap_fitbit_class_nw, cv_fitbit_class_nw = run_fitbit_classification_nw_best(fitbit_class_dict)
perf_comb_reg_nw, shap_comb_reg_nw, cv_comb_reg_nw = run_comb_regression_nw_best(comb_reg_dict)
perf_comb_class_nw, shap_comb_class_nw, cv_comb_class_nw = run_comb_classification_nw_best(comb_class_dict)

In [18]:
# Save the SHAP score dataframes separately as TSV files (without index)
shap_fitbit_reg_nw.to_csv("results/shap_fitbit_reg_nw.tsv", sep="\t", index=False)
shap_fitbit_class_nw.to_csv("results/shap_fitbit_class_nw.tsv", sep="\t", index=False)
shap_comb_reg_nw.to_csv("results/shap_comb_reg_nw.tsv", sep="\t", index=False)
shap_comb_class_nw.to_csv("results/shap_comb_class_nw.tsv", sep="\t", index=False)

### Save train/test performance

In [19]:

# Combine the performance dataframes for regression and classification
best_perf_reg = pd.concat([perf_fitbit_reg, perf_comb_reg, perf_fitbit_reg_nw, perf_comb_reg_nw])
best_perf_class = pd.concat([perf_fitbit_class, perf_comb_class, perf_fitbit_class_nw, perf_comb_class_nw])


# Save the combined performance dataframes as TSV files (without index)
best_perf_reg.to_csv("results/best_perf_reg.tsv", sep="\t", index=False)
best_perf_class.to_csv("results/best_perf_class.tsv", sep="\t", index=False)


### Save validation performance

In [15]:
# Combine the val performance dataframes for regression and classification
best_cv_perf_reg = pd.concat([cv_fitbit_reg, cv_comb_reg, cv_fitbit_reg_nw, cv_comb_reg_nw])
best_cv_perf_class = pd.concat([cv_fitbit_class, cv_comb_class, cv_fitbit_class_nw, cv_comb_class_nw])

# Save the combined val performance dataframes as TSV files (without index)
best_cv_perf_reg.to_csv("results/best_val_perf_reg.tsv", sep="\t", index=False)
best_cv_perf_class.to_csv("results/best_val_perf_class.tsv", sep="\t", index=False)
